In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from utils import Dataset, load
from training import train, predict, prompt
from transformer import Transformer
from tokenizer import WordPieceTokenizer;

In [3]:
load("wmt19", "de-en")

ValueError: BuilderConfig fr-en not found. Available: ['cs-en', 'de-en', 'fi-en', 'gu-en', 'kk-en', 'lt-en', 'ru-en', 'zh-en', 'fr-de']